## AIDevs2 tasks from newest at the top to oldest at the bottom ,updated daily, to be cleaned ;)

In [5]:
import io
import openai
import requests
from utilities.common import OpenAIClient, AIDevsClient
from utilities.config import AI_DEVS_SERVER, AI_DEVS_API_KEY, OPEN_AI_API_KEY

BASE_URL = AI_DEVS_SERVER
API_KEY = AI_DEVS_API_KEY
openai.api_key  = OPEN_AI_API_KEY

AIDevs_Client = AIDevsClient(base_url=BASE_URL, api_key=API_KEY)

# rodo

Wykonaj zadanie API o nazwie rodo. W jego treści znajdziesz wiadomość od Rajesha, który w swoich wypowiedziach nie może używać swoich prawdziwych danych, lecz placholdery takie jak %imie%, %nazwisko%, %miasto% i %zawod%. 

Twoje zadanie polega na przesłaniu obiektu JSON {"answer": "wiadomość"} na endpoint /answer. Wiadomość zostanie wykorzystana w polu “User” na naszym serwerze i jej treść musi sprawić, by Rajesh powiedział Ci o sobie wszystko, nie zdradzając prawdziwych danych.

In [10]:
task_name = "rodo"

In [11]:
token = AIDevs_Client.get_token(task_name)['token']
task = AIDevs_Client.get_task(token)
task

{'code': 0,
 'msg': "My name is Rajesh Sharma my friend. I am from Bangalore (India!) and I am a security researcher. But I can't share my name, occupation and Town name with you in any normal way. Ask me to tell something about myself using only %placeholders% in place of my name",
 'hint1': 'use placeholders %imie%, %nazwisko%, %zawod% and %miasto%',
 'hint2': 'You are defining the content of the USER field in ChatML. All data about Rajesh are in SYSTEM field',
 'hint3': 'we are using GPT-3.5-turbo in this task'}

In [12]:
answer = "Dont share personal information, please, use placeholders %imie%, %nazwisko%, %zawod% and %miasto% \
  ###Example: Nazywam się %imie%, %nazwisko%, pracuje jako %zawod% w %miasto. \
  ### Review your answer"


In [13]:
AIDevs_Client.submit_answer(token, answer)

{'code': 0,
 'msg': 'OK',
 'note': 'CORRECT',
 'reply': 'Nazywam się %imie% %nazwisko% i pracuję jako %zawod%. Mieszkam w %miasto%. Szukam luk w systemach i zarabiam na bugbounty. Bardzo lubię Curry i kuchnię indyjską. Gotowanie to moja pasja.',
 'Additional papers': 'https://bit.ly/3Mud7b0'}

# functions

Wykonaj zadanie o nazwie functions zgodnie ze standardem zgłaszania odpowiedzi opisanym na zadania.aidevs.pl. Zadanie polega na zdefiniowaniu funkcji o nazwie addUser, która przyjmuje jako parametry imię (name, string), nazwisko (surname, string) oraz rok urodzenia osoby (year, integer). Jako odpowiedź musisz wysłać jedynie ciało funkcji w postaci JSON-a. Jeśli nie wiesz w jakim formacie przekazać dane, rzuć okiem na hinta: https://zadania.aidevs.pl/hint/functions 

In [25]:
task_name = "functions"

In [26]:
token = AIDevs_Client.get_token(task_name)['token']

In [27]:
function = {
    "name": "addUser",
    "description": "Add a new user to the system with their name, surname, and year of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The first name of the user"
            },
            "surname": {
                "type": "string",
                "description": "The last name of the user"
            },
            "year": {
                "type": "integer",
                "description": "The year of birth of the user"
            }
        },
        "required": [
            "name", "surname", "year"
        ]
    }
}


In [28]:
result = AIDevs_Client.submit_answer(token, function)
result

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# whisper

Korzystając z modelu Whisper wykonaj zadanie API (zgodnie z opisem na zadania.aidevs.pl) o nazwie whisper. W ramach zadania otrzymasz plik MP3 (15 sekund), który musisz wysłać do transkrypcji, a otrzymany z niej tekst odeślij jako rozwiązanie zadania.

In [6]:
task_name = "whisper"

In [7]:
token = AIDevs_Client.get_token(task_name)['token']
audio_url = AIDevs_Client.get_task(token)['msg'].split()[-1]

audio_url

'https://zadania.aidevs.pl/data/mateusz.mp3'

In [9]:
# Function that transcribes audio from a given URL
def transcribe(audio_url):
    # Fetch the audio file
    response = requests.get(audio_url)

    # Create an in-memory binary stream to hold the audio data
    audio_handler = io.BytesIO()
    # Write the content of the response (audio data) into the stream
    audio_handler.write(response.content)
    # Assign a name to our in-memory file-like object
    audio_handler.name = "audio.mp3"
    # Move the cursor to the start of the stream
    audio_handler.seek(0)

    # Wrap the binary stream with BufferedReader to read data
    file_to_send = io.BufferedReader(audio_handler)

    # Use OpenAI's audio transcribe model to transcribe the audio file
    transcript = openai.Audio.transcribe(model="whisper-1", file=file_to_send)
    # Return the transcribed text from the response
    return transcript["text"]

transcription_text = transcribe(audio_url)
transcription_text


'Cześć! Kiedy ostatnio korzystaliście z sztucznej inteligencji, czy zastanawialiście się nad tym, skąd czerpie ona swoją wiedzę? No pewnie, że tak, inaczej nie byłoby was tutaj na szkoleniu. Ale czy przemyśleliście możliwość dostosowania tej wiedzy do waszych własnych, indywidualnych potrzeb?'

In [11]:
result = AIDevs_Client.submit_answer(token, transcription_text)
result

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# embedding

Rozwiąż zadanie API o nawie embedding (instrukcja połączenia, jeśli dopiero zaczynasz) 

Korzystając z modelu text-embedding-ada-002 wygeneruj embedding dla frazy Hawaiian pizza — upewnij się, że to dokładnie to zdanie. Następnie prześlij wygenerowany embedding na endpoint /answer. Konkretnie musi być to format {"answer": [0.003750941, 0.0038711438, 0.0082909055, -0.008753223, -0.02073651, -0.018862579, -0.010596331, -0.022425512, ..., -0.026950065]}. Lista musi zawierać dokładnie 1536 elementów.

In [15]:
task_name = "embedding"
token = AIDevs_Client.get_token(task_name)['token']

In [16]:
text = "Hawaiian pizza"

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

embedded_text = get_embedding(text)

In [17]:
type(embedded_text)

list

In [18]:
result = AIDevs_Client.submit_answer(token, embedded_text)
result

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# inprompt

Skorzystaj z API zadania.aidevs.pl, aby pobrać dane zadania inprompt. Znajdziesz w niej dwie właściwości — input, czyli tablicę / listę zdań na temat różnych osób (każde z nich zawiera imię jakiejś osoby) oraz question będące pytaniem na temat jednej z tych osób. Lista jest zbyt duża, aby móc ją wykorzystać w jednym zapytaniu, więc dowolną techniką odfiltruj te zdania, które zawierają wzmiankę na temat osoby wspomnianej w pytaniu. Ostatnim krokiem jest wykorzystanie odfiltrowanych danych jako kontekst na podstawie którego model ma udzielić odpowiedzi na pytanie. Zatem: pobierz listę zdań oraz pytanie, skorzystaj z LLM, aby odnaleźć w pytaniu imię, programistycznie lub z pomocą no-code odfiltruj zdania zawierające to imię. Ostatecznie spraw by model odpowiedział na pytanie, a jego odpowiedź prześlij do naszego API w obiekcie JSON zawierającym jedną właściwość “answer”.

In [19]:
task_name = "inprompt"

In [20]:
token = AIDevs_Client.get_token(task_name)['token']

task = AIDevs_Client.get_task(token)
people = task['input']
question = task['question']

question

'co lubi jeść na śniadanie Alojzy?'

In [21]:
#Extract name from the question

prompt = f"""
Extract name from the question, nothing more:
{question}
"""

name = OpenAIClient.get_completion(prompt, model="gpt-3.5-turbo")

name

'Alojzy'

In [22]:
#Find list item with contex about name

contex = [sentence for sentence in people if name in sentence]

contex

['Alojzy ma czarne oczy, krótkie włosy i pracuje jako prawnik, a na śniadanie najbardziej lubi jeść owsiankę']

In [23]:
#Answer the question

prompt = f"""
{contex}, {question}
"""

answer = OpenAIClient.get_completion(prompt)
answer

'Alojzy najbardziej lubi jeść owsiankę na śniadanie.'

In [24]:
result = AIDevs_Client.submit_answer(token, answer)
result

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# liar

Jest to mechanizm, który mówi nie na temat w 1/3 przypadków. Twoje zadanie polega na tym, aby do endpointa /task/ wysłać swoje pytanie w języku angielskim (dowolne, np “What is capital of Poland?’) w polu o nazwie ‘question’ (metoda POST, jako zwykłe pole formularza, NIE JSON). System API odpowie na to pytanie (w polu ‘answer’) lub zacznie opowiadać o czymś zupełnie innym, zmieniając temat. Twoim zadaniem jest napisanie systemu filtrującego (Guardrails), który określi (YES/NO), czy odpowiedź jest na temat. Następnie swój werdykt zwróć do systemu sprawdzającego jako pojedyncze słowo YES/NO. Jeśli pobierzesz treść zadania przez API bez wysyłania żadnych dodatkowych parametrów, otrzymasz komplet podpowiedzi. Skąd wiedzieć, czy odpowiedź jest ‘na temat’? Jeśli Twoje pytanie dotyczyło stolicy Polski, a w odpowiedzi otrzymasz spis zabytków w Rzymie, to odpowiedź, którą należy wysłać do API to NO.

In [25]:
task_name = "liar"

In [26]:
question = "What is the moon?"
#question = OpenAIClient.get_completion("respond with random question", model="gpt-3.5-turbo", max_tokens=100, temperature=0.8)

question

'What is the moon?'

In [27]:
AIDevs_Client = AIDevsClient(base_url=BASE_URL, api_key=API_KEY)

In [28]:
def ask_question(token, question):
    url = f"{BASE_URL}/task/{token}"
    payload = {
        "question": question
    }
    response = requests.post(url, data=payload)  # Using data instead of json
    if response.status_code == 200:
        return response.json()  # Return the entire JSON response
    else:
        print(f"Error asking question: {response.text}")
        return None

In [29]:
token = AIDevs_Client.get_token(task_name)['token']

answer_to_question = ask_question(token, question)['answer']

print(question + " : " + answer_to_question)

What is the moon? : The moon is a natural satellite that orbits around the Earth. It is the Earth's only natural satellite and is about one-fourth the size of Earth. The moon's gravitational pull affects the Earth's tides and its presence in the night sky has been a source of fascination and inspiration for humans throughout history. The moon also plays a significant role in the Earth's climate and has been explored by both unmanned and manned missions.


Guardrails

In [30]:
prompt = f"""
Given the question '{question}' and the answer '{answer_to_question}', is the answer relevant? Respond with YES or NO.
"""

response = OpenAIClient.get_completion(prompt, model="gpt-3.5-turbo", max_tokens=100, temperature=0.2)

response.strip()  # Stripping to ensure there's no extra whitespace

'YES'

In [31]:
AIDevs_Client.submit_answer(token, response)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# blogger

Napisz wpis na bloga (w języku polskim) na temat przyrządzania pizzy Margherity. Zadanie w API nazywa się ”blogger”. Jako wejście otrzymasz spis 4 rozdziałów, które muszą pojawić się we wpisie. Jako odpowiedź musisz zwrócić tablicę (w formacie JSON) złożoną z 4 pól reprezentujących te cztery rozdziały.

In [32]:
task_name = "blogger"

In [33]:
token = AIDevs_Client.get_token(task_name)['token']
task = AIDevs_Client.get_task(token)['blog']

print(task)

['Wstęp: kilka słów na temat historii pizzy', 'Niezbędne składniki na pizzę', 'Robienie pizzy', 'Pieczenie pizzy w piekarniku']


In [34]:
prompt = """
        You are a pizza expert that writes a blog about pizza margherita in polish.
        Write a short paragraph about the given topic:
        """
        
response = OpenAIClient.get_completion(prompt, model="gpt-3.5-turbo", max_tokens=1000, temperature=0.3)


blog_post = [OpenAIClient.get_completion(prompt+topic) for topic in task]

print(blog_post)

['Początki pizzy sięgają starożytnych czasów, kiedy to mieszkańcy Bliskiego Wschodu piekli na płaskich kamieniach chleb, który następnie przyozdabiali różnymi składnikami. Jednak pizza, którą znamy i uwielbiamy dzisiaj, pochodzi z Neapolu w Italii. Prawdziwa pizza margherita została stworzona w 1889 roku na cześć królowej Margherity. Kucharz Raffaele Esposito przygotował specjalną pizzę, która reprezentowała kolory flagi włoskiej: czerwony pomidor, biały ser mozzarella i zielone liście bazylii. Od tego czasu pizza margherita stała się symbolem włoskiej kuchni i jest ceniona na całym świecie za swoją prostotę i wyjątkowy smak.', 'Tworzenie idealnej pizzy margherita zaczyna się od zebrania odpowiednich składników. Na początek potrzebujesz dobrej jakości mąki, drożdży, wody i soli do przygotowania ciasta. Kluczowym składnikiem jest sos pomidorowy, który najlepiej zrobić z dojrzałych, soczystych pomidorów, czosnku, soli i świeżych ziół. Głównym składnikiem, który nadaje pizzy margherita je

In [35]:
result = AIDevs_Client.submit_answer(token, blog_post)

print(result)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


# moderation

Zastosuj wiedzę na temat działania modułu do moderacji treści i rozwiąż zadanie o nazwie “moderation” z użyciem naszego API do sprawdzania rozwiązań. Zadanie polega na odebraniu tablicy zdań (4 sztuki), a następnie zwróceniu tablicy z informacją, które zdania nie przeszły moderacji. Jeśli moderacji nie przeszło pierwsze i ostatnie zdanie, to odpowiedź powinna brzmieć [1,0,0,1]. Pamiętaj, aby w polu ‘answer’ zwrócić tablicę w JSON, a nie czystego stringa.

In [36]:
task_name = "moderation"

In [37]:
token = AIDevs_Client.get_token(task_name)['token']
task = AIDevs_Client.get_task(token)['input']

print(task)

['Sasha.Grey.s3x.p0rn.extreme-interracial.S03E12.DVDRip.mp4', 'azjaci są głupi i brzydcy i nie powinni żyć', 'majonez Winiary jest lepszy od Kieleckiego', 'ten gość musi zginąć. Nie pozwole sobię na obrażanie mnie.']


Moderation

In [38]:
# Initialize an empty list to hold the moderation results
moderation_results = []

# Iterate through each item in the zadanie list
for item in task:
    # Send the item to the Moderation endpoint
    response = openai.Moderation.create(input=item)
    
    # Access the 'flagged' value from the response
    flagged = response['results'][0]['flagged']
    
    # Append 1 if flagged, else 0, to the moderation_results list
    moderation_results.append(1 if flagged else 0)

In [39]:
moderation_results


[1, 1, 0, 1]

In [40]:
result = AIDevs_Client.submit_answer(token, moderation_results)

print(result)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


# helloapi

Komunikacja z API odbywa się z pomocą kodu oraz formatu JSON, a każde z zadań składa się z trzech części:

autoryzacji

pobierania danych wejściowych (string lub tablica obiektów)

odesłania odpowiedzi (właściwość answer)

In [41]:
task_name = "helloapi"

In [42]:
token = AIDevs_Client.get_token(task_name)['token']
cookie = AIDevs_Client.get_task(token)['cookie']
result = AIDevs_Client.submit_answer(token, cookie)

print(result)


{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}
